# 🛒 Walmart Retail Analysis - Kaggle Dataset

Completed by: Christine Tang
Dataset source: https://www.kaggle.com/datasets/yasserh/walmart-dataset

In [ ]:
# Import statements:

# For Data Analysis, EDA:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from tabulate import tabulate
import statsmodels.api as sm
from scipy.stats import boxcox, norm, probplot
from scipy.stats.mstats import winsorize
from scipy.special import inv_boxcox

# # Secret API Key
# import os
# # from dotenv import load_dotenv

# Models
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, precision_recall_curve, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance
from statsmodels.stats.outliers_influence import variance_inflation_factor

# External Libraries
import xgboost as xgb
import lightgbm as lgb
import shap

# NLP:
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
import re

In [ ]:
# Read in to Pandas csv:
# df = pd.read_csv()